# Import libraries and load the datasets

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

customers_df = pd.read_csv('Customers.csv')
products_df = pd.read_csv('Products.csv')
transactions_df = pd.read_csv('Transactions.csv')

# 1. Data Preprocessing and Merging the Datasets
Merge the transaction data with customer and product data

In [2]:
merged_df = transactions_df.merge(customers_df, on='CustomerID', how='left')  # Merge with Customers
merged_df = merged_df.merge(products_df, on='ProductID', how='left')  # Merge with Products


# 2. Feature Engineering for Transaction History
Aggregating purchase information per customer

In [5]:
cp_history = merged_df.groupby('CustomerID').agg(
    total_spent=pd.NamedAgg(column='TotalValue', aggfunc='sum'),
    avg_transaction_value=pd.NamedAgg(column='TotalValue', aggfunc='mean'),
    total_transactions=pd.NamedAgg(column='TransactionID', aggfunc='nunique')
).reset_index()

# Feature for the most purchased product
cate_spend = merged_df.groupby(['CustomerID', 'Category'])['TotalValue'].sum().unstack(fill_value=0)
cate_spend = cate_spend.reset_index()

# Merge transaction aggregated data with the category data
cprofile = pd.merge(cp_history, cate_spend, on='CustomerID')


# 3. Normalizing Features

In [6]:
sc = StandardScaler()
sc_features = sc.fit_transform(cprofile.drop('CustomerID', axis=1))

# 4. Calculate Cosine Similarity
We will compute similarity using cosine similarity between customers' feature vectors

In [7]:
cose_simi = cosine_similarity(sc_features)


# 5. Function to Get Lookalike Recommendations

In [12]:
def get_lookalikes(customer_id, top_n=3):
    customer_index = cprofile[cprofile['CustomerID'] == customer_id].index[0]
    similarity_scores = cose_simi[customer_index]

    # Get sorted indices based on similarity (excluding the customer itself)
    similar_customers = similarity_scores.argsort()[::-1]
    similar_customers = similar_customers[similar_customers != customer_index][:top_n]

    # Get customer IDs and similarity scores
    similar_customers_ids = cprofile.iloc[similar_customers]['CustomerID'].values
    similarity_scores = similarity_scores[similar_customers]

    return list(zip(similar_customers_ids, similarity_scores))

# 6. Generate Lookalike Recommendations for Customers C0001 to C0020

In [13]:
lookalike_map = {}
for customer_id in ['C0001', 'C0002', 'C0003', 'C0004', 'C0005', 'C0006', 'C0007', 'C0008', 'C0009', 'C0010',
                    'C0011', 'C0012', 'C0013', 'C0014', 'C0015', 'C0016', 'C0017', 'C0018', 'C0019', 'C0020']:
    lookalike_map[customer_id] = get_lookalikes(customer_id)

# 7. Create the Lookalike.csv file

In [14]:
lookalike_df = pd.DataFrame([
    {'CustomerID': cust_id, 'Lookalikes': str(lookalikes)}
    for cust_id, lookalikes in lookalike_map.items()
])

lookalike_df.to_csv('Lookalike.csv', index=False)
print("Lookalike recommendations for customers C0001 - C0020 have been saved to 'Lookalike.csv'.")

Lookalike recommendations for customers C0001 - C0020 have been saved to 'Lookalike.csv'.


In [17]:
df = pd.read_csv('Lookalike.csv');
print(df.head(5))

  CustomerID                                         Lookalikes
0      C0001  [('C0069', 0.9682358410432497), ('C0091', 0.92...
1      C0002  [('C0036', 0.8785858823455555), ('C0055', 0.86...
2      C0003  [('C0166', 0.9269596398494403), ('C0005', 0.90...
3      C0004  [('C0075', 0.9804638599143275), ('C0065', 0.92...
4      C0005  [('C0197', 0.9790240655576301), ('C0166', 0.92...
